In [ ]:
import sys

sys.path.append("lib/")

import os
import glob
import json
import jsonpickle

import TFilter as f
import TEventFilter as ef
from TDataType import TDataType as tdt
from TWeatherType import TWeatherType as twt
from TWeekDay import TWeekDay as twd
from TMap import TMap
from TPlotter import TPlotter
from TReader import TReader

In [ ]:
# load data
filepaths = glob.glob(os.path.join("../../../DP/filtered/", "*.csv"))
zones_file = "data/zones.json"
 
LIMIT = 0  # number or 0 for no limit

reader = TReader(filepaths)

# Load TDFs
ltdfs = reader.read(LIMIT)

print(f"Loaded:\t{len(ltdfs)}")

In [ ]:
# settings
T_Line = ("*", "*", "*")
T_Date = ("01.10.2022", "02.10.2022")
T_Time = ("00:00:00", "23:59:59")
T_Vehicle = "7525"

T_Data_type = tdt.ALL
T_Weather_type = twt.ALL
T_Week_day = twd.ALL

T_Map_events = True
T_Plot_events = False

T_Events_filter = True
T_Events_stop = True
T_Events_zero = True
T_Events_diff = 20

# depos zones
with open(zones_file, "r") as file:
    filter_zones = json.load(file)

# filter all ltdfs and data
data = f.filter_tdfs(ltdfs, T_Line, T_Date, T_Week_day, T_Vehicle)
print(f"TDFs:\t\t{len(data)}")

for d in data:
    d.filter_rows(T_Data_type)
    d.filter_zones(filter_zones)

# detect and filter events
events = ef.detect_events(data)
print(f"All events:\t{len(events)}")

events = ef.filter_time(events, T_Time)
print(f"In time:\t{len(events)}")

if T_Weather_type != T_Weather_type.ALL:
    events = ef.filter_weather(events, T_Weather_type)
print(f"In weather:\t{len(events)}")

if T_Events_filter:
    if T_Events_diff:
        events = ef.filter_diff(events, T_Events_diff)
    if T_Events_zero:
        events = ef.filter_zero(events)
    if T_Events_stop:
        events = ef.filter_embs(events)
    print(f"Filtered:\t{len(events)}")

In [ ]:
m = TMap([48.1700, 17.1077], 13)

m.add_filter_zones(filter_zones)

# plot into map
if T_Map_events:
    m.add_events(events)
else:
    m.add_data(data)

m.save(False)
m.save_png()

print(m.n)
m.m

In [ ]:
p = TPlotter(events)

if T_Plot_events:
    p.plot()
    p.save()

In [ ]:
# Serialize event objects
json_data = jsonpickle.encode(events, unpicklable=False, indent=4)

# Write JSON data to a file
with open("data/output.json", "w") as json_file:
    json_file.write(json_data)

# write event csv
import pandas
data_dicts = [obj.__dict__ for obj in events]
edf = pandas.DataFrame(data_dicts).drop(columns=["points"])
edf.to_csv("data/events.csv")